In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import pickle
import matplotlib.pyplot as plt

# Datos Tasa de Incidencia Covid

In [2]:
basepath = "/media/jonathan/JHON/Tesis/DatosTesis/Data_Covid19/covid19FEB_JUL.csv"
data = pd.read_csv(basepath, sep=';',dtype={"municipio_distrito": str,
                                        "tasa_incidencia_acumulada_ultimos_14dias": str,
                                        "tasa_incidencia_acumulada_total": str},index_col='fecha_informe',parse_dates=True, usecols=[0,1,3], encoding='latin-1')

data['TasaIncidenciaAcumulada_Ultimos14dias'] = [int(value.split(",",1)[0]) for value in data['tasa_incidencia_acumulada_ultimos_14dias']]
#data.dtypes

basepath = "/media/jonathan/JHON/Tesis/DatosTesis/Data_Covid19/covid19JUL_2022.csv"
data2 = pd.read_csv(basepath, sep=';',dtype={"municipio_distrito": str,
                                        "tasa_incidencia_acumulada_ultimos_14dias": str,
                                        "tasa_incidencia_acumulada_total": str},index_col='fecha_informe',parse_dates=True, usecols=[0,1,5], encoding='latin-1')

data2['TasaIncidenciaAcumulada_Ultimos14dias'] = [int(value.split(",",1)[0]) for value in data2['tasa_incidencia_acumulada_ultimos_14dias']]
data2.head(5)

,municipio_distrito,tasa_incidencia_acumulada_ultimos_14dias,TasaIncidenciaAcumulada_Ultimos14dias
fecha_informe,,,
2022-03-29 11:51:00,Madrid-Retiro,"301,6",301
2022-03-29 11:51:00,Madrid-Salamanca,"266,5",266
2022-03-29 11:51:00,Madrid-Centro,"273,97",273
2022-03-29 11:51:00,Madrid-Arganzuela,"241,57",241
2022-03-29 11:51:00,Madrid-Chamartín,"265,67",265


# Tablas Tasa de Incidencia Covid

In [3]:
table = pd.pivot_table(data, values=["TasaIncidenciaAcumulada_Ultimos14dias"], index=["fecha_informe"], columns=["municipio_distrito"], aggfunc={"TasaIncidenciaAcumulada_Ultimos14dias":"first"})
table.head(5)

table2 = pd.pivot_table(data2, values=["TasaIncidenciaAcumulada_Ultimos14dias"], index=["fecha_informe"], columns=["municipio_distrito"], aggfunc={"TasaIncidenciaAcumulada_Ultimos14dias":"first"})
table2.head(5)

table = table.astype(np.float64)
table2 = table2.astype(np.float64)

#Quitamos los niveles sobrantes del nivel superior y reiniciamos index - Solo se hace una vez de lo contrario genera problema
table.columns =[str(s2) for (s1,s2) in table.columns.tolist()]
table_Final = table.reset_index()
table_Final.set_index('fecha_informe', inplace = True)

table2.columns =[str(s2) for (s1,s2) in table2.columns.tolist()]
table2_Final = table2.reset_index()
table2_Final.set_index('fecha_informe', inplace = True)

#table.dtypes
#table.iloc[120:127,97:115]
table2_Final.head(2)

,Ajalvir,Alameda del Valle,Alcalá de Henares,Alcobendas,Alcorcón,Aldea del Fresno,Algete,Alpedrete,Ambite,Anchuelo,...,Villamanta,Villamantilla,Villanueva de Perales,Villanueva de la Cañada,Villanueva del Pardillo,Villar del Olmo,Villarejo de Salvanés,Villaviciosa de Odón,Villavieja del Lozoya,Zarzalejo
fecha_informe,,,,,,,,,,,,,,,,,,,,,
2020-05-26 07:00:00,21.0,0.0,24.0,9.0,26.0,70.0,4.0,20.0,0.0,0.0,...,0.0,69.0,0.0,27.0,17.0,0.0,40.0,14.0,0.0,180.0
2020-06-02 07:00:00,0.0,0.0,18.0,7.0,24.0,35.0,38.0,13.0,0.0,0.0,...,0.0,69.0,0.0,51.0,17.0,0.0,13.0,25.0,0.0,0.0


# Procesamiento datos desplazamiento por trayecto

In [82]:
def df_origen(origen,destino):     
    
    ######################PICKLE###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names2.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)

    df = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Referencia2.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)    
    
    ######################PICKLE 1###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento1.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df1 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento1.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 2###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento2.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df2 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento2.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 3###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento3.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df3 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento3.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 4###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento4.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df4 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento4.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 5###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento5.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df5 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento5.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 6###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento6.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df6 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento6.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    #Filtramos el destino
    columns_names_df = df.columns   
    names_1 = list(columns_names_df)   
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)            
    df_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Referencia2.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)    
    
    #Cambiamos el nombre de las columnas
    lines_names = {"19024":"Fuera_Madrid","2800601":"Fuera_Madrid","2806504":"Fuera_Madrid","2807901":"Centro","2807902":"Arganzuela","2807903":"Retiro","2807904":"Salamanca","2807905":"Chamartín","2807906":"Tetuán","2807907":"Chamberí","2807908":"El_Pardo","2807909":"Moncloa-Aravaca","2807910":"Latina","2807911":"Carabanchel","2807912":"Usera","2807913":"Puente_de_Vallecas","2807914":"Moratalaz","2807915":"Ciudad_Lineal","2807916":"Hortaleza","2807917":"Villaverde","2807918":"Villa_de_Vallecas","2807919":"Vicálvaro","2807920":"San Blas-Canillejas","2812301":"Fuera_Madrid","2807921":"Barajas"}            
    df_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df_Final['Fecha'] = df_Final.index.date
    df_Final['Año'] = df_Final.index.year                                
    df_Final['Mes'] = df_Final.index.month                               
    df_Final['ID_Dia'] = df_Final.index.dayofweek
    df_Final['Hora'] = df_Final.index.hour
    df_Final['WeekNo'] = df_Final.index.isocalendar().week
    df_Final[lines_names[origen]] = table_Final["Madrid"+"-"+lines_names[origen]].loc['2020-02-14':'2020-03-16']
          
    
    #Filtramos el destino
    columns_names_df = df1.columns   
    names_1 = list(columns_names_df)    
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)
    df1_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento1.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)
    
    #Cambiamos el nombre de las columnas    
    df1_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df1_Final['Fecha'] = df1_Final.index.date
    df1_Final['Año'] = df1_Final.index.year                                
    df1_Final['Mes'] = df1_Final.index.month                               
    df1_Final['ID_Dia'] = df1_Final.index.dayofweek
    df1_Final['Hora'] = df1_Final.index.hour
    df1_Final['WeekNo'] = df1_Final.index.isocalendar().week
    df1_Final[lines_names[origen]] = table_Final["Madrid"+"-"+lines_names[origen]].loc['2020-03-17':'2020-05-31']    
    

    #Filtramos el destino
    columns_names_df = df2.columns   
    names_1 = list(columns_names_df)    
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)
    df2_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento2.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)
    
    #Cambiamos el nombre de las columnas    
    df2_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df2_Final['Fecha'] = df2_Final.index.date
    df2_Final['Año'] = df2_Final.index.year                                
    df2_Final['Mes'] = df2_Final.index.month                               
    df2_Final['ID_Dia'] = df2_Final.index.dayofweek
    df2_Final['Hora'] = df2_Final.index.hour
    df2_Final['WeekNo'] = df2_Final.index.isocalendar().week   
    df2_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2020-06-01':'2020-07-31']
    
    
    #Filtramos el destino
    columns_names_df = df3.columns   
    names_1 = list(columns_names_df)    
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)
    df3_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento3.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)
    
    #Cambiamos el nombre de las columnas    
    df3_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df3_Final['Fecha'] = df3_Final.index.date
    df3_Final['Año'] = df3_Final.index.year                                
    df3_Final['Mes'] = df3_Final.index.month                               
    df3_Final['ID_Dia'] = df3_Final.index.dayofweek
    df3_Final['Hora'] = df3_Final.index.hour
    df3_Final['WeekNo'] = df3_Final.index.isocalendar().week
    df3_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2020-08-01':'2020-10-31']
    
    
    #Filtramos el destino
    columns_names_df = df4.columns   
    names_1 = list(columns_names_df)    
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)
    df4_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento4.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)
    
    #Cambiamos el nombre de las columnas    
    df4_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df4_Final['Fecha'] = df4_Final.index.date
    df4_Final['Año'] = df4_Final.index.year                                
    df4_Final['Mes'] = df4_Final.index.month                               
    df4_Final['ID_Dia'] = df4_Final.index.dayofweek
    df4_Final['Hora'] = df4_Final.index.hour
    df4_Final['WeekNo'] = df4_Final.index.isocalendar().week   
    df4_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2020-11-01':'2020-12-31']
    
    
    #Filtramos el destino
    columns_names_df = df5.columns   
    names_1 = list(columns_names_df)   
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)            
    df5_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento5.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)    
    
    #Cambiamos el nombre de las columnas    
    df5_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df5_Final['Fecha'] = df5_Final.index.date
    df5_Final['Año'] = df5_Final.index.year                                
    df5_Final['Mes'] = df5_Final.index.month                               
    df5_Final['ID_Dia'] = df5_Final.index.dayofweek
    df5_Final['Hora'] = df5_Final.index.hour
    df5_Final['WeekNo'] = df5_Final.index.isocalendar().week    
    df5_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2021-01-01':'2021-01-31']
    

    #Filtramos el destino
    columns_names_df = df6.columns   
    names_1 = list(columns_names_df)   
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)            
    df6_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento6.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)        
    
    #Cambiamos el nombre de las columnas    
    df6_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df6_Final['Fecha'] = df6_Final.index.date
    df6_Final['Año'] = df6_Final.index.year                                
    df6_Final['Mes'] = df6_Final.index.month                               
    df6_Final['ID_Dia'] = df6_Final.index.dayofweek
    df6_Final['Hora'] = df6_Final.index.hour
    df6_Final['WeekNo'] = df6_Final.index.isocalendar().week  
    df6_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2021-02-01':'2021-05-09']
    
    
    return df_Final, df1_Final, df2_Final, df3_Final, df4_Final, df5_Final, df6_Final, listaDst

origen = "2807905"
destino = "2807901"
df_Final, df1_Final, df2_Final, df3_Final, df4_Final, df5_Final, df6_Final, listaDst = df_origen(origen,destino)
df_Final.tail(2)

,Chamartín-Centro,Fecha,Año,Mes,ID_Dia,Hora,WeekNo,Chamartín
ds,,,,,,,,
2020-03-16 22:00:00,55.801,2020-03-16,2020,3,0,22,12,NaN
2020-03-16 23:00:00,21.306,2020-03-16,2020,3,0,23,12,NaN


# Nota: No está colocando los datos de covid 19 en la tabla debido a que la tabla de covid entrega datos por Ej a las 11:59:00, los datos de la tabla de abajo tiene los datos cada hora y no tiene en cuenta datos con los minutos. 

In [104]:
df6_Final[2210:2260]

,Chamartín-Centro,Fecha,Año,Mes,ID_Dia,Hora,WeekNo,Chamartín
ds,,,,,,,,
2021-05-04 02:00:00,20.872,2021-05-04,2021,5,1,2,18,NaN
2021-05-04 03:00:00,56.876,2021-05-04,2021,5,1,3,18,NaN
2021-05-04 04:00:00,98.224,2021-05-04,2021,5,1,4,18,NaN
2021-05-04 05:00:00,97.779,2021-05-04,2021,5,1,5,18,NaN
2021-05-04 06:00:00,189.870,2021-05-04,2021,5,1,6,18,NaN
2021-05-04 07:00:00,344.872,2021-05-04,2021,5,1,7,18,NaN
2021-05-04 08:00:00,534.829,2021-05-04,2021,5,1,8,18,NaN
2021-05-04 09:00:00,497.847,2021-05-04,2021,5,1,9,18,NaN
2021-05-04 10:00:00,469.123,2021-05-04,2021,5,1,10,18,NaN


In [97]:
table2_Final.iloc[40:100,88:200]

,Madrid-Chamartín,Madrid-Chamberí,Madrid-Ciudad Lineal,Madrid-Fuencarral-El Pardo,Madrid-Hortaleza,Madrid-Latina,Madrid-Moncloa-Aravaca,Madrid-Moratalaz,Madrid-Puente de Vallecas,Madrid-Retiro,...,Villamanta,Villamantilla,Villanueva de Perales,Villanueva de la Cañada,Villanueva del Pardillo,Villar del Olmo,Villarejo de Salvanés,Villaviciosa de Odón,Villavieja del Lozoya,Zarzalejo
fecha_informe,,,,,,,,,,,,,,,,,,,,,
2021-03-02 10:51:00,289.0,340.0,287.0,229.0,311.0,260.0,378.0,392.0,322.0,251.0,...,156.0,138.0,0.0,331.0,454.0,248.0,68.0,391.0,0.0,422.0
2021-03-09 11:44:00,259.0,297.0,268.0,219.0,251.0,221.0,283.0,251.0,262.0,189.0,...,273.0,416.0,127.0,242.0,337.0,298.0,122.0,344.0,0.0,482.0
2021-03-16 10:45:00,279.0,273.0,275.0,230.0,212.0,213.0,286.0,208.0,240.0,200.0,...,156.0,416.0,255.0,251.0,232.0,298.0,177.0,237.0,0.0,542.0
2021-03-23 12:16:00,285.0,264.0,236.0,237.0,184.0,222.0,275.0,179.0,246.0,201.0,...,78.0,138.0,63.0,303.0,232.0,149.0,122.0,204.0,0.0,180.0
2021-03-30 11:53:00,331.0,329.0,260.0,237.0,239.0,227.0,316.0,203.0,294.0,274.0,...,195.0,0.0,255.0,442.0,250.0,49.0,149.0,326.0,395.0,361.0
2021-04-06 11:56:00,333.0,375.0,282.0,265.0,275.0,240.0,324.0,275.0,289.0,293.0,...,352.0,0.0,383.0,415.0,442.0,0.0,245.0,405.0,790.0,542.0
2021-04-13 11:51:00,359.0,436.0,371.0,324.0,355.0,298.0,372.0,344.0,351.0,324.0,...,230.0,134.0,439.0,334.0,695.0,144.0,174.0,337.0,381.0,288.0
2021-04-20 13:22:00,421.0,515.0,451.0,378.0,417.0,356.0,475.0,365.0,450.0,413.0,...,76.0,202.0,690.0,416.0,615.0,240.0,134.0,465.0,763.0,173.0
2021-04-27 11:54:00,458.0,521.0,422.0,371.0,366.0,372.0,440.0,333.0,409.0,424.0,...,115.0,202.0,376.0,416.0,396.0,192.0,147.0,436.0,763.0,57.0
